In [38]:
import os 
import logging 

logging.basicConfig(level=logging.INFO)

import numpy as np 
import pandas as pd 

In [40]:
# 1. Cross validation of dune data and brean's data 
df_dune = pd.read_csv("./beanft-dune.csv", header=None, names=["account", "ids", "count_nfts"], index_col=False)
df_brean = pd.read_csv("./beanft-brean.csv")
assert len(df_dune) == 396
assert len(df_brean) == 918
# 1.A. Unique wallets receiving NFT's 
df_dune.account = df_dune.account.apply(lambda acct: acct.replace("\\x", '0x').lower())
df_brean.account = df_brean.account.str.lower()
dune_unique_accounts = set(np.unique(df_dune.account))
brean_unique_accounts = set(np.unique(df_brean.account))
intersect_unique_accounts = set(dune_unique_accounts).intersection(set(brean_unique_accounts))
print(f"Num unique accounts dune: {len(dune_unique_accounts)}")
print(f"Num unique accounts brean: {len(brean_unique_accounts)}")
print(f"Num unique accounts in common: {len(intersect_unique_accounts)}")
print(f"Difference dune: {dune_unique_accounts - intersect_unique_accounts}")
print(f"Difference dune: {brean_unique_accounts - intersect_unique_accounts}")
assert len(dune_unique_accounts) == len(brean_unique_accounts) == 396

Num unique accounts dune: 396
Num unique accounts brean: 396
Num unique accounts in common: 396
Difference dune: set()
Difference dune: set()


/var/folders/2g/r74bpyq56fv9vbvc0fphvn480000gr/T/ipykernel_22335/3559424067.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_dune = pd.read_csv("./beanft-dune.csv", header=None, names=["account", "ids", "count_nfts"], index_col=False)


In [41]:
import json 
with open("dune_results_2.json", 'r') as f: 
    dune_results_2 = json.loads(f.read())['query_results']
df_dune_2 = pd.DataFrame([d['data'] for d in dune_results_2])
df_dune_2.account = df_dune_2.account.apply(lambda acct: acct.replace("\\x", '0x').lower())
df_dune_2.head()

,account,count_nfts,sum_value
0,0x01914d6e47657d6a7893f84fc84660dc5aec08b6,1,10000
1,0x02009370ff755704e9acbd96042c1ab832d6067e,18,95689
2,0x02491d37984764d39b99e4077649dcd349221a62,1,3000
3,0x026a8f79db3d678e5881a9e42cc50e7ac88aaf25,1,4975
4,0x027be82bf7895db5fc1fea5696117e875bbcc0de,1,20250


In [42]:
# Ensures totals match up 
assert df_dune_2.sum_value.sum() == df_brean.value.sum() == 14292152
# Ensures nft counts match up 
brean_nft_counts = {
    list(t)[1]: list(t)[2] for t in 
    df_brean[['account', 'value']].groupby("account"
        ).count(
        ).reset_index(
        ).rename(columns={"value": "count"}
        ).itertuples()
}
dune_nft_counts = {
    list(t)[1]: list(t)[2] for t in 
    df_dune_2.itertuples()
}
assert brean_nft_counts == dune_nft_counts
assert len(brean_nft_counts) == len(dune_nft_counts) == 396
assert np.sum(list(brean_nft_counts.values())) == np.sum(list(dune_nft_counts.values()))